# Huggingface 토크나이저 테스트
pre-trained model : monologg/koelectra-base-v3-discriminator"

github : [https://github.com/monologg/KoELECTRA](https://github.com/monologg/KoELECTRA)

### Ref.
[Huggingface 토크나이저 설명](https://huffon.github.io/2020/07/05/tokenizers/)

In [3]:
import pandas as pd

save_dir = ('../data/조선경제_2019.09.01_2021.08.31.csv')
df = pd.read_csv(save_dir)
df['body_prep'] = df['body'].str.replace(pat=r'[^A-Za-z0-9가-힣]', repl= r' ', regex=True)
df['body_prep'].iloc[0]

'                                1일 오후 10시 31분께 대구 달서구의 한 4층짜리 모텔 건물 3층 객실에서 불이 났다   화재가 발생하자 건물 안에 있던 손님 30여명이 급하게 대피했으나 불이 시작된 객실 안에 있던 40대 남성 A씨가 연기를 흡입해 병원으로 옮겨졌다   또 객실 안 침대 등이 불에 탔다   소방당국은 소방차 24대와 소방관 50명을 투입해 화재 발생 10여분 뒤인 오후 10시 45분께 진화를 완료했다   대구소방본부 관계자는  화재 원인 등을 조사하고 있다 고 밝혔다    연합뉴스         '

In [4]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForTokenClassification
from transformers import ElectraModel, ElectraTokenizer, ElectraTokenizerFast

In [5]:
pt_model_name_list = [
    'monologg/koelectra-base-v3-discriminator"',
]

def init_tokenizer(model_name):
    return ElectraTokenizerFast.from_pretrained(
            "monologg/koelectra-base-v3-discriminator",
            use_fast=True,
            add_prefix_space=False if "roberta" not in model_name else True,
        )

In [13]:
for pt_model_name in pt_model_name_list:
    print(f"INIT {pt_model_name} tokenizer")
    tokenizer = init_tokenizer(pt_model_name)
    print()
    
    indexed_tokens = []
    for index in range(len(df)):
        text = df['body_prep'].iloc[index]
        print(f"{pt_model_name} will tokenize '{text}'")
        
        tokenized_inputs = tokenizer(text, return_offsets_mapping=True)
        print(f"{pt_model_name} tokenized_inputs")
        print(tokenized_inputs)
        print()

        # print(f"{pt_model_name} tokenized_inputs.word_ids")
        # print(tokenized_inputs.word_ids(batch_index=0))
        # print()

        batch_encoded = tokenizer.batch_encode_plus(
            [text],
            max_length=512,
            truncation=True,
            padding="max_length",
            return_offsets_mapping=True,
        )
        # print(f"{pt_model_name} batch_encoded:")
        # print(batch_encoded)
        # print()

        encoded = tokenizer.encode(
            text,
            max_length=512,
            truncation=True,
            padding="max_length",
            return_offsets_mapping=True,
            return_tensors="pt")
        print(f"{pt_model_name} encoded:")
        print(encoded)
        print()
        indexed_tokens.append(encoded)

        print(f"{pt_model_name} decoded:")
        decoded = tokenizer.decode(encoded)
        print(decoded)
        print()

        batch_decoded = tokenizer.batch_decode(
            batch_encoded["input_ids"], skip_special_tokens=True
        )
        tokens = tokenizer.tokenize(batch_decoded[0])
        # tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text)))
        print(f"{pt_model_name} tokenizes tokens:")
        print(len(tokens))
        print(tokens)

        offsets_mapping = batch_encoded["offset_mapping"][0]

        batch_decoded = tokenizer.batch_decode(
            batch_encoded["input_ids"], skip_special_tokens=True
        )
        print(f"{pt_model_name} batch_decoded")
        print(batch_decoded)
        tokenized_outputs = tokenizer.tokenize(batch_decoded[0])
        print(f"{pt_model_name} tokenized outputs")
        print(tokenized_outputs)

        print("********")
        print()
        print()
        break

INIT monologg/koelectra-base-v3-discriminator" tokenizer

monologg/koelectra-base-v3-discriminator" will tokenize '                                1일 오후 10시 31분께 대구 달서구의 한 4층짜리 모텔 건물 3층 객실에서 불이 났다   화재가 발생하자 건물 안에 있던 손님 30여명이 급하게 대피했으나 불이 시작된 객실 안에 있던 40대 남성 A씨가 연기를 흡입해 병원으로 옮겨졌다   또 객실 안 침대 등이 불에 탔다   소방당국은 소방차 24대와 소방관 50명을 투입해 화재 발생 10여분 뒤인 오후 10시 45분께 진화를 완료했다   대구소방본부 관계자는  화재 원인 등을 조사하고 있다 고 밝혔다    연합뉴스         '
monologg/koelectra-base-v3-discriminator" tokenized_inputs
{'input_ids': [2, 21, 4366, 6640, 6242, 4114, 7036, 4473, 5253, 6676, 19397, 4050, 4234, 3757, 24, 4370, 4353, 4108, 18101, 7017, 23, 4370, 14081, 4073, 4129, 2803, 4007, 2246, 4176, 8537, 4070, 6442, 4279, 4195, 7017, 3081, 4073, 3249, 4820, 9602, 6332, 4044, 4282, 4007, 14462, 4325, 13139, 4398, 6742, 2803, 4007, 6273, 4880, 14081, 3081, 4073, 3249, 4820, 6625, 4289, 7173, 37, 4561, 4070, 7144, 4110, 16852, 4151, 6615, 10749, 22782, 4176, 2489, 14081, 3081, 9860, 6569, 2803, 4073, 3604, 4176, 8310, 4403, 4113, 

TypeError: 'list' object cannot be interpreted as an integer

In [9]:
doc_list = df['body_prep'].to_list()

In [14]:
encoded_input = tokenizer(
            doc_list,
            max_length=512,
            truncation=True,
            padding="max_length",
            return_offsets_mapping=True,
            return_tensors="pt")

In [15]:
model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
import torch
with torch.no_grad():
    model_output = model(input_ids = encoded_input["input_ids"])

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:79] data. DefaultCPUAllocator: not enough memory: you tried to allocate 57178324992 bytes.

In [1]:
indexed_tokens

NameError: name 'indexed_tokens' is not defined